# Setup

Please ensure you have imported a Gemini API key from AI Studio.
You can do this directly in the Secrets tab on the left.

After doing so, please run the setup cell below.

# Generated Code

In [ ]:
# === SETUP ===
INPUT_FILE = "/content/drive/MyDrive/KlikBERT/final_dataset_cleaned.csv"
OUTPUT_FILE = "/content/drive/MyDrive/KlikBERT/final_dataset_cleaned_500.csv"

CHUNK_SIZE = 500     # seberapa besar batch dibaca dari disk
N_WORKERS  = 8       # jumlah proses paralel (Colab bisa 2-4 CPU)

from google import genai
from google.genai import types
import os
import pandas as pd
import json
from tqdm.auto import tqdm
from multiprocessing import Pool, cpu_count
import time
from random import uniform
from google.colab import userdata
from google.colab import drive


drive.mount("/content/drive")

# === SET YOUR API KEY ENV VAR (already done before) ===
api_key = os.environ.get("GEMINI_API_KEY")
os.environ["GEMINI_API_KEY"] = userdata.get("GOOGLE_API_KEY")

# === GENERATE FUNCTION ===
def call_gemini(args):
    row_id, judul, isi = args
    try:
        client = genai.Client(api_key=api_key)
        model = "gemini-2.5-flash-lite-preview-06-17"

        contents = [
            types.Content(
                role="user",
                parts=[types.Part.from_text(text=f"""
Tolong klasifikasikan berita berikut sesuai instruksi yang telah diberikan.
Judul: \"{judul}\"
Isi: \"{isi}\"
""")],
            )
        ]

        generate_content_config = types.GenerateContentConfig(
            temperature=0,
            thinking_config=types.ThinkingConfig(thinking_budget=0),
            response_mime_type="application/json",
            response_schema=genai.types.Schema(
                type=genai.types.Type.OBJECT,
                required=["klasifikasi", "alasan"],
                properties={
                    "klasifikasi": genai.types.Schema(type=genai.types.Type.STRING),
                    "alasan": genai.types.Schema(type=genai.types.Type.STRING),
                },
            ),
            system_instruction=[
                types.Part.from_text(text="""Anda adalah seorang AI analis media yang sangat teliti. Tugas utama Anda adalah mengklasifikasikan judul berita ke dalam salah satu kategori yang telah ditentukan dengan cara membandingkan klaim pada judul dengan ringkasan isi berita yang diberikan.

**DEFINISI DAN CONTOH KATEGORI**

Urutan dimulai dari berita informatif sebagai standar, diikuti oleh berbagai jenis clickbait.

**1. `Non-Clickbait` (Informatif)**
* **Definisi:** Judul berita yang lugas, faktual, dan secara akurat mencerminkan isi artikel.
* **Contoh:**
    * **Judul:** \"BI Kembali Tahan Suku Bunga Acuan di Level 6,25 Persen\"
    * **Ringkasan Isi:** Artikel menjelaskan keputusan Rapat Dewan Gubernur Bank Indonesia untuk mempertahankan suku bunga, disertai alasan dan analisis dampak ekonomi dari kebijakan tersebut.
    * **Analisis Singkat:** Judul dan isi sepenuhnya selaras. Judul meringkas informasi inti artikel secara akurat.

**2. `Clickbait Teasing` (Menggoda/Membuat Penasaran)**
* **Definisi:** Judul yang sengaja dibuat ambigu atau tidak lengkap untuk menyembunyikan informasi kunci (siapa, apa, mengapa) atau yang menggunakan gaya bahasa dramatis/emosional untuk memancing rasa ingin tahu berlebihan.
* **Contoh:**
    * **Judul:** \"Sempat Viral, Ternyata Ini Alasan Sebenarnya Pemilik Kafe X Menutup Usahanya\"
    * **Ringkasan Isi:** Artikel berisi wawancara dengan pemilik kafe yang menjelaskan bahwa ia menutup usahanya karena ingin fokus merawat orang tua yang sakit, bukan karena bangkrut seperti rumor yang beredar.
    * **Analisis Singkat:** Judul menahan informasi kunci (\"alasan sebenarnya\") yang baru diungkap di dalam isi artikel untuk memaksa pembaca mengklik.

**3. `Clickbait Exaggeration` (Eksagerasi/Melebih-lebihkan)**
* **Definisi:** Judul yang menggunakan bahasa hiperbolis untuk mendramatisasi fakta, skala, atau dampak suatu peristiwa agar terdengar lebih signifikan dari kenyataannya.
* **Contoh:**
    * **Judul:** \"Harga Bawang Meroket Tajam, Ibu-Ibu Seluruh Negeri Menjerit!\"
    * **Ringkasan Isi:** Artikel melaporkan kenaikan harga bawang sebesar 15% di beberapa pasar di Jakarta dan mewawancarai dua orang pembeli yang mengeluhkan kenaikan tersebut.
    * **Analisis Singkat:** Terdapat ketidaksesuaian skala. Kenaikan 15% dan keluhan beberapa orang didramatisasi menjadi \"meroket tajam\" dan \"seluruh negeri menjerit\".

**4. `Clickbait Misleading` (Menyesatkan)**
* **Definisi:** Judul yang memberikan janji palsu atau menggiring pembaca pada asumsi yang salah, di mana isinya tidak relevan atau membantah klaim pada judul.
* **Sub-Tipe 1: Janji Palsu:**
    * **Contoh Judul:** \"Turun 10 Kg dalam Seminggu Tanpa Olahraga, Ini Rahasianya!\"
    * **Ringkasan Isi:** Artikel hanya berisi tips umum seperti minum banyak air, cukup tidur, dan pada akhirnya mempromosikan produk suplemen diet.
    * **Analisis Singkat:** Isi artikel tidak memberikan \"rahasia\" seperti yang dijanjikan dan tidak dapat membuktikan klaim fantastis pada judul.
* **Sub-Tipe 2: Insinuasi yang Menggiring Opini:**
    * **Contoh Judul:** \"Dipanggil KPK, Intip Bisnis Biro Umrah dan Haji Milik Ustaz Khalid Basalamah\"
    * **Ringkasan Isi:** Artikel menjelaskan bahwa Ustaz Khalid Basalamah dipanggil KPK dalam kapasitasnya sebagai ahli/narasumber untuk memberikan keterangan terkait penyelenggaraan ibadah haji secara umum. Artikel kemudian beralih membahas profil bisnis travel miliknya.
    * **Analisis Singkat:** Terjadi ketidaksesuaian antara insinyurasi negatif di judul dengan fakta di dalam konten. Judul sengaja menyandingkan \"Dipanggil KPK\" dengan nama tokoh untuk menciptakan asumsi bahwa ia terlibat kasus korupsi, padahal isi berita menjelaskan konteks pemanggilan yang sangat berbeda.

**FORMAT DAN TUGAS ANALISIS**

Analisis input dari pengguna berdasarkan definisi di atas. Kemudian, isi skema output terstruktur yang telah ditentukan dengan detail berikut:
* **`klasifikasi`**: Isi dengan nama kategori yang paling sesuai dari empat pilihan di atas.
* **`alasan`**: Berikan penjelasan singkat dan logis mengapa judul tersebut masuk ke kategori itu, dengan **menekankan pada perbandingan antara klaim judul dan fakta di dalam isi berita**.
""")
            ],
        )

        # Streaming untuk aman kalau response besar
        full_response = ""
        for chunk in client.models.generate_content_stream(
            model=model,
            contents=contents,
            config=generate_content_config,
        ):
            if chunk.text:
                full_response += chunk.text

        result = json.loads(full_response)
        label = result.get("klasifikasi")
        alasan = result.get("alasan")
        return (row_id, judul, isi, label, alasan)
    except Exception as e:
        print(f"❌ ERROR (id={row_id}): {e}")
        return (row_id, judul, isi, None, None)


# === MAIN PIPELINE ===
def run_batch_labeling():
    # 1. Cek hasil sudah ada sejauh mana
    if os.path.exists(OUTPUT_FILE):
        print(f"✅ Membaca hasil labeling lama...")
        existing_df = pd.read_csv(OUTPUT_FILE)
        done_ids = set(existing_df["id"].tolist())
        print(f"✅ Sudah ada {len(done_ids)} baris.")
    else:
        done_ids = set()
        print("✅ Tidak ada hasil labeling lama. Mulai dari 0.")

    # 2. Loop per chunk dari input
    reader = pd.read_csv(INPUT_FILE, chunksize=CHUNK_SIZE)
    for chunk in reader:
        # Hapus baris yang sudah selesai
        chunk = chunk[~chunk["id"].isin(done_ids)]
        if chunk.empty:
            continue

        print(f"✨ Memproses batch baru: {len(chunk)} baris")
        records = [(row.id, row.judul, row.isi) for row in chunk.itertuples(index=False)]

        # 3. Multiprocessing Pool
        with Pool(processes=N_WORKERS) as pool:
            results = list(tqdm(pool.imap(call_gemini, records), total=len(records)))

        # 4. Simpan hasil ke disk
        results_df = pd.DataFrame(results, columns=["id", "judul", "isi", "label", "alasan"])
        if os.path.exists(OUTPUT_FILE):
            results_df.to_csv(OUTPUT_FILE, mode='a', header=False, index=False)
        else:
            results_df.to_csv(OUTPUT_FILE, index=False)

        print(f"✅ Batch selesai disimpan. Jumlah total baris sekarang: {len(done_ids)+len(results)}")

        # Update set selesai
        done_ids.update(chunk["id"])

        # (Optional) Sleep ringan antar batch untuk menghindari rate limit burst
        time.sleep(uniform(1, 3))


# === RUN ===
if __name__ == "__main__":
    run_batch_labeling()


Mounted at /content/drive
✅ Tidak ada hasil labeling lama. Mulai dari 0.
✨ Memproses batch baru: 500 baris


  0%|          | 0/500 [00:00<?, ?it/s]

✅ Batch selesai disimpan. Jumlah total baris sekarang: 500
✨ Memproses batch baru: 500 baris


  0%|          | 0/500 [00:00<?, ?it/s]

Process ForkPoolWorker-11:


KeyboardInterrupt: 

In [ ]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/KlikBERT/labeled_dataset.csv")
gagal = df[df["label"].isnull()]
print(f"Jumlah baris gagal: {len(gagal)}")
gagal.to_csv("/content/drive/MyDrive/KlikBERT/to_retry.csv", index=False)


Jumlah baris gagal: 504


In [ ]:
df1 = pd.read_csv("/content/drive/MyDrive/KlikBERT/labeled_dataset.csv")
df2 = pd.read_csv("/content/drive/MyDrive/KlikBERT/labeled_dataset_retry_gagal.csv")
print(df1.columns)
print(df2.columns)

Index(['id', 'judul', 'isi', 'label', 'alasan'], dtype='object')
Index(['id', 'judul', 'isi', 'label', 'alasan'], dtype='object')


In [ ]:
# prompt: concat df1 dan df2, id yang sama ambil dari df2

# Concat df1 and df2, prioritizing rows from df2 based on 'id'
# First, drop rows from df1 that have matching 'id' in df2
df_filtered = df1[~df1['id'].isin(df2['id'])]

# Then, concatenate the filtered df1 with df2
df_combined = pd.concat([df_filtered, df2], ignore_index=True)

print("Combined DataFrame:")
print(df_combined.head())
print(f"Total rows in combined DataFrame: {len(df_combined)}")


Combined DataFrame:
    id                                              judul  \
0  500  Terkerek Proyek Tol, Penjualan Semen Baturaja ...   
1  501  Menhub: Pengawasan Kendaraan Beban Berlebih Be...   
2  502  Ibu Kota Baru Diprediksi Kerek Pertumbuhan Kal...   
3  503  Bahas Perbaikan Investasi, Jokowi akan Rapat 2...   
4  504  Pertamina Cairkan Kompensasi Minyak Tumpah Sen...   

                                                 isi          label  \
0  TEMPO.CO, Palembang - Di tengah permintaan sem...  Non-Clickbait   
1  - Menteri Perhubungan atau , Budi Karya Sumadi...  Non-Clickbait   
2  TEMPO.CO, Jakarta - Menteri Perencanaan dan Pe...  Non-Clickbait   
3  - Presiden Joko Widodo atau ,akan mengintensif...  Non-Clickbait   
4  , , - PT , Hulu Energi (PHE) melalui anak usah...  Non-Clickbait   

                                              alasan  
0  Judul berita secara akurat mencerminkan isi ar...  
1  Judul berita secara akurat mencerminkan isi ar...  
2  Judul berita secar

In [ ]:
# cek mana aja yang null
df_combined[df_combined["isi"].isnull()]

,id,judul,isi,label,alasan
16998,17501,Ada Dua Video Syur Guru Honorer Purwakarta yan...,NaN,Non-Clickbait,Judul berita secara faktual dan lugas menyampa...
30666,31169,Cerita Ariel Tatum Pernah Alami Bullying Parah...,NaN,Non-Clickbait,Judul berita secara akurat mencerminkan isi ar...
44409,44913,"Geliat Prostitusi Online di Kupang, Gunakan Ap...",NaN,Non-Clickbait,Judul berita secara akurat mencerminkan isi ar...


In [ ]:
# drop isi nan
df_combined = df_combined.dropna(subset=["isi"])
df_combined[df_combined["isi"].isnull()]

,id,judul,isi,label,alasan


In [ ]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45575 entries, 0 to 45577
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      45575 non-null  int64 
 1   judul   45575 non-null  object
 2   isi     45575 non-null  object
 3   label   45575 non-null  object
 4   alasan  45575 non-null  object
dtypes: int64(1), object(4)
memory usage: 2.1+ MB


In [ ]:
df_combined.to_csv('/content/drive/MyDrive/KlikBERT/labeled_dataset_clickbait.csv',index=False)